In [42]:
import pandas as pd
import numpy as np

In [43]:
gdp_data = pd.read_csv("data/gdp/API_NY.GDP.MKTP.KD_DS2_en_csv_v2_5728927.csv",skiprows=4)
pop_data = pd.read_csv("data/population/API_SP.POP.TOTL_DS2_en_csv_v2_5695140/API_SP.POP.TOTL_DS2_en_csv_v2_5695140.csv",skiprows=4)
pop_dens_data = pd.read_csv("data/population/API_EN.POP.DNST_DS2_en_csv_v2_5729014/API_EN.POP.DNST_DS2_en_csv_v2_5729014.csv",skiprows=4)
urban_area_data = pd.read_csv("data/urban/API_AG.LND.TOTL.UR.K2_DS2_en_csv_v2_5735377/API_AG.LND.TOTL.UR.K2_DS2_en_csv_v2_5735377.csv",skiprows=4)
urban_pop_perc_data = pd.read_csv("data/urban/API_SP.URB.TOTL.IN.ZS_DS2_en_csv_v2_5734574/API_SP.URB.TOTL.IN.ZS_DS2_en_csv_v2_5734574.csv",skiprows=4)

In [44]:
pop_data

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54608.0,55811.0,56682.0,57475.0,58178.0,58782.0,...,103594.0,104257.0,104874.0,105439.0,105962.0,106442.0,106585.0,106537.0,106445.0,NaN
1,Africa Eastern and Southern,AFE,"Population, total",SP.POP.TOTL,130692579.0,134169237.0,137835590.0,141630546.0,145605995.0,149742351.0,...,583651101.0,600008424.0,616377605.0,632746570.0,649757148.0,667242986.0,685112979.0,702977106.0,720839314.0,NaN
2,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8622466.0,8790140.0,8969047.0,9157465.0,9355514.0,9565147.0,...,32716210.0,33753499.0,34636207.0,35643418.0,36686784.0,37769499.0,38972230.0,40099462.0,41128771.0,NaN
3,Africa Western and Central,AFW,"Population, total",SP.POP.TOTL,97256290.0,99314028.0,101445032.0,103667517.0,105959979.0,108336203.0,...,397855507.0,408690375.0,419778384.0,431138704.0,442646825.0,454306063.0,466189102.0,478185907.0,490330870.0,NaN
4,Angola,AGO,"Population, total",SP.POP.TOTL,5357195.0,5441333.0,5521400.0,5599827.0,5673199.0,5736582.0,...,27128337.0,28127721.0,29154746.0,30208628.0,31273533.0,32353588.0,33428486.0,34503774.0,35588987.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Kosovo,XKX,"Population, total",SP.POP.TOTL,947000.0,966000.0,994000.0,1022000.0,1050000.0,1078000.0,...,1812771.0,1788196.0,1777557.0,1791003.0,1797085.0,1788878.0,1790133.0,1786038.0,1761985.0,NaN
262,"Yemen, Rep.",YEM,"Population, total",SP.POP.TOTL,5542459.0,5646668.0,5753386.0,5860197.0,5973803.0,6097298.0,...,27753304.0,28516545.0,29274002.0,30034389.0,30790513.0,31546691.0,32284046.0,32981641.0,33696614.0,NaN
263,South Africa,ZAF,"Population, total",SP.POP.TOTL,16520441.0,16989464.0,17503133.0,18042215.0,18603097.0,19187194.0,...,54729551.0,55876504.0,56422274.0,56641209.0,57339635.0,58087055.0,58801927.0,59392255.0,59893885.0,NaN
264,Zambia,ZMB,"Population, total",SP.POP.TOTL,3119430.0,3219451.0,3323427.0,3431381.0,3542764.0,3658024.0,...,15737793.0,16248230.0,16767761.0,17298054.0,17835893.0,18380477.0,18927715.0,19473125.0,20017675.0,NaN


In [45]:
# inter(extra)-polate missing urban area values
test = urban_area_data[urban_area_data.columns[24:-1]].T
test.index = pd.to_numeric(test.index, errors='coerce')

urban_area_data[urban_area_data.columns[24:-1]] = test.interpolate(method="slinear", fill_value="extrapolate", limit_direction="both").T


In [46]:
# find urban density from other datasets
urban_pop_dens = pop_data.copy() 
urban_pop_dens[urban_pop_dens.columns[4:-1]] = urban_pop_dens[urban_pop_dens.columns[4:-1]] * urban_pop_perc_data[urban_pop_perc_data.columns[4:-1]]/(urban_area_data[urban_area_data.columns[4:-1]]*100)
urban_pop_dens


,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Aruba,ABW,"Population, total",SP.POP.TOTL,NaN,NaN,NaN,NaN,NaN,NaN,...,461.214576,461.708800,462.184641,462.618159,463.055222,463.492583,462.644183,461.173976,459.708828,NaN
1,Africa Eastern and Southern,AFE,"Population, total",SP.POP.TOTL,NaN,NaN,NaN,NaN,NaN,NaN,...,3149.613058,3228.244571,3304.905303,3380.866725,3462.399695,3546.273022,3632.330000,3718.509072,3805.064990,NaN
2,Afghanistan,AFG,"Population, total",SP.POP.TOTL,NaN,NaN,NaN,NaN,NaN,NaN,...,2073.400086,2096.200278,2109.493121,2131.512889,2156.834096,2185.467664,2221.845196,2254.972648,2283.642434,NaN
3,Africa Western and Central,AFW,"Population, total",SP.POP.TOTL,NaN,NaN,NaN,NaN,NaN,NaN,...,3500.987366,3587.229313,3674.983063,3764.530463,3854.793161,3945.830189,4038.293476,4131.058034,4224.498265,NaN
4,Angola,AGO,"Population, total",SP.POP.TOTL,NaN,NaN,NaN,NaN,NaN,NaN,...,5816.182061,5965.845469,6118.437905,6273.584039,6427.721443,6581.970192,6731.905465,6878.914568,7024.823441,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Kosovo,XKX,"Population, total",SP.POP.TOTL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
262,"Yemen, Rep.",YEM,"Population, total",SP.POP.TOTL,NaN,NaN,NaN,NaN,NaN,NaN,...,5397.800056,5523.783892,5649.221599,5775.826328,5902.075500,6028.982540,6152.806501,6269.448232,6389.971756,NaN
263,South Africa,ZAF,"Population, total",SP.POP.TOTL,NaN,NaN,NaN,NaN,NaN,NaN,...,2347.563794,2395.954075,2418.443876,2426.779622,2455.511226,2486.181525,2515.334601,2538.964016,2558.610282,NaN
264,Zambia,ZMB,"Population, total",SP.POP.TOTL,NaN,NaN,NaN,NaN,NaN,NaN,...,3172.540161,3274.989556,3379.732927,3487.214500,3596.817605,3708.327966,3820.946365,3933.792122,4047.082741,NaN


In [47]:
# inter(extra)-polate missing population density values
test = pop_dens_data[pop_dens_data.columns[4:]].T
test.index = pd.to_numeric(test.index, errors='coerce')

pop_dens_data[pop_dens_data.columns[4:]] = test.interpolate(method="slinear", fill_value="extrapolate", limit_direction="both").T


In [48]:
# melt dataframes
gdp_melt = pd.melt(gdp_data,id_vars=["Country Code"],value_vars=gdp_data.columns[4:-1].values,var_name="Year",value_name="GDP")
pop_melt = pd.melt(pop_data,id_vars=["Country Code"],value_vars=pop_data.columns[4:-1].values,var_name="Year",value_name="POP")
pop_dens_melt = pd.melt(pop_dens_data,id_vars=["Country Code"],value_vars=pop_dens_data.columns[4:-1].values,var_name="Year",value_name="POP-DENS")
urban_pop_perc_melt = pd.melt(urban_pop_perc_data,id_vars=["Country Code"],value_vars=urban_pop_perc_data.columns[4:-1].values,var_name="Year",value_name="URBAN-POP-PERC")
urban_pop_dens_melt = pd.melt(urban_pop_dens,id_vars=["Country Code"],value_vars=urban_pop_dens.columns[4:-1].values,var_name="Year",value_name="URBAN-POP-DENS")

In [49]:
#merge dataframes
syn_data = gdp_melt
syn_data = syn_data.merge(pop_melt,on=["Country Code","Year"])
syn_data = syn_data.merge(pop_dens_melt,on=["Country Code","Year"])
syn_data= syn_data.merge(urban_pop_perc_melt,on=["Country Code","Year"])
syn_data= syn_data.merge(urban_pop_dens_melt,on=["Country Code","Year"])

In [50]:
syn_data["GDP_rising"] = 1
syn_data["GDP_falling"] = 0
for i,row in syn_data.iterrows():
    if int(row["Year"]) != 1960 and not np.isnan(row["GDP"]):
        if row["GDP"]/row["POP"] < syn_data.loc[(syn_data["Country Code"]==row["Country Code"])&(syn_data["Year"]==str(int(row["Year"])-1))]["GDP"].values[0]/syn_data.loc[(syn_data["Country Code"]==row["Country Code"])&(syn_data["Year"]==str(int(row["Year"])-1))]["POP"].values[0]:
            syn_data.at[i,"GDP_rising"] = 0
            syn_data.at[i,"GDP_falling"] = 1

syn_data

,Country Code,Year,GDP,POP,POP-DENS,URBAN-POP-PERC,URBAN-POP-DENS,GDP_rising,GDP_falling
0,ABW,1960,NaN,54608.0,305.222222,50.776000,NaN,1,0
1,AFE,1960,1.540941e+11,130692579.0,8.955968,14.563810,NaN,1,0
2,AFG,1960,NaN,8622466.0,13.202755,8.401000,NaN,1,0
3,AFW,1960,1.058619e+11,97256290.0,10.742515,14.705391,NaN,1,0
4,AGO,1960,NaN,5357195.0,4.300366,10.435000,NaN,1,0
...,...,...,...,...,...,...,...,...,...
16487,XKX,2021,7.912105e+09,1786038.0,NaN,NaN,NaN,1,0
16488,YEM,2021,NaN,32981641.0,62.544086,38.546000,6269.448232,1,0
16489,ZAF,2021,3.532584e+11,59392255.0,49.062146,67.847000,2538.964016,1,0
16490,ZMB,2021,2.449592e+10,19473125.0,26.197491,45.192000,3933.792122,1,0


In [51]:
vehicle_stock_data = pd.read_csv("data/vehicle stock/consolidated_vehicle_stock_data_cleaned.csv",index_col=0)
vehicle_stock_data.columns
vehicle_stock_melt = pd.melt(vehicle_stock_data,id_vars=["Country code"],value_vars=vehicle_stock_data.columns[2:-1].values,var_name="Year",value_name="VEHICLE-STOCK")

syn_data = syn_data.merge(vehicle_stock_melt,left_on=["Country Code","Year"],right_on=["Country code","Year"])


In [52]:
syn_data["VEHICLE-STOCK-LAST"] = np.nan
for i, row in syn_data.iterrows():
    row_pre = syn_data.loc[(syn_data["Country Code"]==row["Country Code"])& (syn_data.Year == str(int(row.Year)-1))]
    if not row_pre.empty:
        #print(row_pre)

        #print(syn_data.loc[(syn_data["Country Code"]==row["Country Code"])& (syn_data.Year == row.Year),"VEHICLE-STOCK-LAST"])
        #print(row_pre.values[0])
        syn_data.loc[(syn_data["Country Code"]==row["Country Code"])& (syn_data.Year == row.Year),"VEHICLE-STOCK-LAST"] = row_pre["VEHICLE-STOCK"].values[0]


In [53]:
print("before vehicle stock: ",len(syn_data))
syn_data = syn_data.drop(syn_data[np.isnan(syn_data["VEHICLE-STOCK"])].index)
print("before vehicle stock last: ",len(syn_data))

syn_data = syn_data.drop(syn_data[np.isnan(syn_data["VEHICLE-STOCK-LAST"])].index)
print("before GDP: ",len(syn_data))

syn_data = syn_data.drop(syn_data[np.isnan(syn_data["GDP"])].index)
print("before URBAN-POP-DENS: ",len(syn_data))

syn_data = syn_data.drop(syn_data[np.isinf(syn_data["URBAN-POP-DENS"])].index)
print("final: ",len(syn_data))


syn_data = syn_data.drop("Country code",axis=1)
syn_data["GDP-PER-CAP"] = syn_data["GDP"]/syn_data["POP"]/1000
syn_data["VEHICLE-STOCK-PER-CAP"] = syn_data["VEHICLE-STOCK"]/syn_data["POP"]*1000
syn_data["VEHICLE-STOCK-LAST-PER-CAP"] = syn_data["VEHICLE-STOCK-LAST"]/syn_data["POP"]*1000



before vehicle stock:  6880
before vehicle stock last:  2236
before GDP:  2006
before URBAN-POP-DENS:  1983
final:  1978


In [54]:
syn_data.to_csv("data/consolidated/consolidated_data.csv")